<a href="https://colab.research.google.com/github/adampotton/Cognitive_AI_CW/blob/main/Q2A_LSTM_BI_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Adpating the LSTM model cost function to tackle sparsity

In [ ]:
! git clone https://github.com/neurogym/neurogym.git
%cd neurogym/
! pip install -e .

Cloning into 'neurogym'...
remote: Enumerating objects: 11100, done.
remote: Counting objects: 100% (1001/1001), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 11100 (delta 928), reused 898 (delta 898), pack-reused 10099 (from 1)
Receiving objects: 100% (11100/11100), 8.17 MiB | 10.97 MiB/s, done.
Resolving deltas: 100% (8335/8335), done.
/content/neurogym
Obtaining file:///content/neurogym
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.4/696.4 kB 8.1 MB/s eta 0:00:00


In [ ]:
import neurogym as ngym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import time

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:396: UserWarning: WARN: The `registry.all` method is deprecated. Please use `registry.values` instead.
  logger.warn(


### Define a simple LSTM model

In [ ]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_lstm_layers = 2):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_lstm_layers) # n LSTM layers
        self.fc = nn.Linear(hidden_size, output_size) # Fully connected layer

    def forward(self, x):
        lstm_output, _ = self.lstm(x)
        out = self.fc(lstm_output)
        return out, lstm_output

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Creating dataset and adjusting parameters

In [ ]:
config = {
    'dt': 200, # Timestep parameter
    'hidden_size': 32, # Hidden size for your LSTM
    'batch_size': 32, # Batch size for training
    'seq_len': 50, # Sequence length for input data
    'envid': 'ReadySetGo-v0', # Task name
    'gain': 2, # Custom gain
    'prod_margin': 10, # Custom production margin
}

env_kwargs = {
    'dt': config['dt'], # Assing timestep parameter
    'gain': config['gain'],  # Controls the measure that the agent has to produce
    'prod_margin': config['prod_margin'], # Controls the interval around the ground truth production time within which the agent receives proportional reward
}
config['env_kwargs'] = env_kwargs

dataset = ngym.Dataset(config['envid'], env_kwargs=config['env_kwargs'], batch_size=config['batch_size'], seq_len=config['seq_len']) # Generate dataset
env = dataset.env

inputs, target = dataset() # Assing inputs and targets
inputs = torch.from_numpy(inputs).type(torch.float)

input_size = env.observation_space.shape[0] # Find dimensions for data
output_size = env.action_space.n

print('Input has shape (SeqLen, Batch, Dim) =', inputs.shape)
print('Target has shape (SeqLen, Batch) =', target.shape)

Input has shape (SeqLen, Batch, Dim) = torch.Size([100, 16, 3])
Target has shape (SeqLen, Batch) = (100, 16)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


### Training the model

In [ ]:
iter_steps = 3000 # Training loops
report_freq = 50 # How often a report on is returned
l1_beta = 0.75 # Sparsity control

net = LSTMNet(input_size, config['hidden_size'], output_size, num_lstm_layers = 1) # Create an instance of the LSTM

def train_model(net, dataset, iter_steps, report_freq, l1_beta):

    optimizer = optim.Adam(net.parameters(), lr=0.0003) # Adam optimiser
    criterion = nn.CrossEntropyLoss() # Loss funciton

    running_loss = 0
    running_acc = 0
    start_time = time.time() # Start training timer

    for i in range(iter_steps): # Loop over training batches
        inputs, labels = dataset() # Generate a set of data
        inputs = torch.from_numpy(inputs).type(torch.float)
        labels = torch.from_numpy(labels.flatten()).type(torch.long)

        optimizer.zero_grad() # Reset gradients
        output, _ = net(inputs)
        output = output.view(-1, output_size)


        l1_reg = 0 # Initialise the L1 Regularisation term
        for param in net.parameters(): # Extract all weight matrices from the network
            if param.requires_grad: # ensure that regularisation is only applied to trainable parts of the model
                l1_reg += torch.sum(torch.abs(param)) # Sum the matrices
        loss = criterion(output, labels) + l1_beta * l1_reg # Define new modified loss funciton

        loss.backward()

        optimizer.step()  # Update

        batch_acc = (torch.argmax(output, dim=1) == labels).sum().item() / labels.shape[0] # Current batch accuracy
        running_loss += loss.item()
        running_acc += batch_acc

        if i % report_freq == report_freq - 1:
            running_loss /= report_freq
            running_acc /= report_freq  # average accuracy over the last 100 batches
            print('Step {}, Loss {:0.4f}, Accuracy {:0.4f}, Time {:0.1f}s'.format(
                i+1, running_loss, running_acc, time.time() - start_time))
            running_loss = 0 # Reset metrics for next report
            running_acc = 0
    return net

net = train_model(net, dataset, iter_steps, report_freq, l1_beta) # Call the training function

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Step 100, Loss 32.1535, Accuracy 0.9281, Time 1.3s
Step 200, Loss 5.2581, Accuracy 0.9621, Time 2.5s
Step 300, Loss 5.2538, Accuracy 0.9254, Time 4.3s
Step 400, Loss 5.2488, Accuracy 0.9436, Time 6.0s


### Measuring the sparsity

In [ ]:
sparsity = 0
total_params = 0
for param in net.parameters():
    if param.requires_grad:
        total_params += param.numel()
        sparsity += torch.sum(param <= 0.001).item()

sparsity_ratio = sparsity / total_params
print('Sparsity ratio:', sparsity_ratio)

Sparsity ratio: 0.5041649312786339


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Peformance Metrics

In [ ]:
avg_losses = []
avg_accuracies = []
report_freq = 20
for i in range(report_freq - 1, len(losses), report_freq):
    avg_losses.append(sum(losses[i-report_freq+1:i+1]) / report_freq)
    avg_accuracies.append(sum(accuracies[i-report_freq+1:i+1]) / report_freq)

iterations = range(report_freq, len(losses) + 1, report_freq)

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

# Plot Loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(iterations, avg_losses, label='Loss', color='blue')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(iterations, avg_accuracies, label='Accuracy', color='green')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.ylim(0.8, 1.05)
plt.title('Training Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
env = dataset.env # Reset environment
env.reset(no_step=True)

perf = 0 # Initialize loggin vars
activity_dict = {}
trial_infos = {}

num_trial = 200
for i in range(num_trial):

    trial_info = env.new_trial() # New trial
    ob, gt = env.ob, env.gt # Observation and groud-truth of this trial
    inputs = torch.from_numpy(ob[:, np.newaxis, :]).type(torch.float)

    action_pred, rnn_activity = net(inputs) # Run network for one trial

    action_pred = action_pred.detach().numpy()[:, 0, :] # Compute performance
    choice = np.argmax(action_pred[-1, :]) # Final choice at final time step
    correct = choice == gt[-1]

    rnn_activity = rnn_activity[:, 0, :].detach().numpy() # Record activity
    activity_dict[i] = rnn_activity
    trial_infos[i] = trial_info  # Record trial infos
    trial_infos[i].update({'correct': correct})

for i in range(20):
    print('Trial ', i, trial_infos[i])

print('Average performance', np.mean([val['correct'] for val in trial_infos.values()]))

Trial  0 {'measure': 800.0, 'gain': 2, 'production': 1600.0, 'correct': True}
Trial  1 {'measure': 1000.0, 'gain': 2, 'production': 2000.0, 'correct': True}
Trial  2 {'measure': 800.0, 'gain': 2, 'production': 1600.0, 'correct': True}
Trial  3 {'measure': 1000.0, 'gain': 2, 'production': 2000.0, 'correct': True}
Trial  4 {'measure': 800.0, 'gain': 2, 'production': 1600.0, 'correct': True}
Trial  5 {'measure': 1200.0, 'gain': 2, 'production': 2400.0, 'correct': True}
Trial  6 {'measure': 1000.0, 'gain': 2, 'production': 2000.0, 'correct': True}
Trial  7 {'measure': 800.0, 'gain': 2, 'production': 1600.0, 'correct': True}
Trial  8 {'measure': 1200.0, 'gain': 2, 'production': 2400.0, 'correct': True}
Trial  9 {'measure': 1000.0, 'gain': 2, 'production': 2000.0, 'correct': True}
Trial  10 {'measure': 1400.0, 'gain': 2, 'production': 2800.0, 'correct': True}
Trial  11 {'measure': 1000.0, 'gain': 2, 'production': 2000.0, 'correct': True}
Trial  12 {'measure': 1200.0, 'gain': 2, 'production':